# Mount Google Drive Storage

In [2]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


# Unzip Training Dataset ZIP File

In [3]:
import zipfile

dataset_zip_file_path = "/content/drive/MyDrive/face_mask_detection_dataset.zip"
dataset_zip_file_extraction_path = "/content/face_mask_detection_dataset"

with zipfile.ZipFile(dataset_zip_file_path, "r") as f:
  f.extractall(dataset_zip_file_extraction_path)

# Check Dataset ZIP File By Listing The Folders Inside It

In [4]:
import os
print(os.listdir(dataset_zip_file_extraction_path + "/data"))

['without_mask', 'with_mask']


# Split Dataset In To Training & Testing(Validation)

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = (224, 224)
batch_size = 32
data_path = dataset_zip_file_extraction_path + "/data"

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

training_data_generator = datagen.flow_from_directory(
    data_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_data_generator = datagen.flow_from_directory(
    data_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.


# Fine Tune Existing MobileNetV2 Model As A Base Model For Face Mask Detection

In [6]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,210 (9.24 MB)

 Trainable params: 164,226 (641.51 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

# Train The Model By 10 Epochs

In [7]:
epochs = 10

history = model.fit(training_data_generator, epochs=epochs, validation_data=validation_data_generator)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
 28/189 ━━━━━━━━━━━━━━━━━━━━ 16s 104ms/step - accuracy: 0.7139 - loss: 0.5903

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


189/189 ━━━━━━━━━━━━━━━━━━━━ 41s 159ms/step - accuracy: 0.8506 - loss: 0.3375 - val_accuracy: 0.9781 - val_loss: 0.0743
Epoch 2/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 17s 89ms/step - accuracy: 0.9736 - loss: 0.0747 - val_accuracy: 0.9801 - val_loss: 0.0516
Epoch 3/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 17s 89ms/step - accuracy: 0.9801 - loss: 0.0543 - val_accuracy: 0.9841 - val_loss: 0.0480
Epoch 4/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 17s 87ms/step - accuracy: 0.9882 - loss: 0.0390 - val_accuracy: 0.9848 - val_loss: 0.0414
Epoch 5/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 17s 92ms/step - accuracy: 0.9894 - loss: 0.0322 - val_accuracy: 0.9848 - val_loss: 0.0396
Epoch 6/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 16s 87ms/step - accuracy: 0.9916 - loss: 0.0261 - val_accuracy: 0.9854 - val_loss: 0.0385
Epoch 7/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 18s 94ms/step - accuracy: 0.9922 - loss: 0.0226 - val_accuracy: 0.9887 - val_loss: 0.0334
Epoch 8/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 17s 88ms/step - accuracy: 0.9915 - loss: 0.0235 - val_accura

# Save The Model In H5 Format

In [8]:
model.save("/content/drive/MyDrive/face_mask_detector.h5")

# Make Prediction On An Unseen Data Also Print The Prediction Classes

In [9]:
import numpy as np
from tensorflow.keras.preprocessing import image

test_image_file_path = "/content/drive/MyDrive/test_1.jpg"
test_image = image.load_img(test_image_file_path, target_size=(224, 224))

image_array = image.img_to_array(test_image)
image_array = np.expand_dims(image_array, axis=0)
image_array /= 255.0

prediction = model.predict(image_array)
print("Prediction", prediction)
print("Prediction Class", training_data_generator.class_indices)

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Prediction [[9.9965215e-01 3.4779709e-04]]
Prediction Class {'with_mask': 0, 'without_mask': 1}


#Convert The Model To Tensorflow Lite Format For Mobile & Edge Devices

In [10]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/face_mask_detector.h5")
tflite_converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = tflite_converter.convert()

with open("/content/drive/MyDrive/face_mask_detector.tflite", "wb") as f:
  f.write(tflite_model)

Saved artifact at '/tmp/tmpta3pr8vi'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  135896262375760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135896259795600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135896259800400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135896259798480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135896259802320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135896259800976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135896259792720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135896259801936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135896259801168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135896259799056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135896259803472

# Download The Trained Tensorflow & Converted TensorflowLite Models

In [12]:
from google.colab import files

files.download("/content/drive/MyDrive/face_mask_detector.h5")
files.download("/content/drive/MyDrive/face_mask_detector.tflite")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>